In [ ]:
import pandas as pd

In [ ]:
cic = pd.read_csv('cicids2017_preprocessed.csv')

In [ ]:
ton = pd.read_csv('type_before_pre_ton.csv')

In [ ]:
del ton['Unnamed: 0'] 

In [ ]:
# 공통 feature 제외하고 제거

In [ ]:
cic_p = cic.drop(['Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Destination Port', 'Timestamp', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio', 'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward', 'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min', 'External IP'], axis= 'columns')

In [ ]:
ton_p = ton.drop(['ts', 'src_port', 'dst_port', 'service', 'conn_state', 'missed_bytes', 'dns_qclass', 'dns_qtype', 'dns_rcode', 'dns_AA', 'dns_RD', 'dns_RA', 'dns_rejected', 'ssl_version', 'ssl_cipher', 'ssl_resumed', 'ssl_established', 'ssl_subject', 'ssl_issuer', 'http_trans_depth', 'http_method', 'http_version', 'http_request_body_len', 'http_response_body_len', 'http_status_code', 'http_orig_mime_types', 'http_resp_mime_types', 'weird_name', 'weird_addl', 'weird_notice','label'], axis= 'columns')

In [ ]:
# 공통 feature 이름 매핑

In [ ]:
ton_p.rename(columns = {'duration':'Flow Duration','proto':'Protocol','src_bytes':'Fwd Packet Length Mean','dst_bytes':'Bwd Packet Length Mean','src_pkts':'Total Fwd Packets','src_ip_bytes':'Fwd Packets/s','dst_pkts':'Total Bwd Packets','dst_ip_bytes':'Bwd Packets/s','type':'Label'},inplace=True)
cic_p.rename(columns = {'Total Backward Packets':'Total Bwd Packets'},inplace=True)

In [ ]:
# feature 타입 통일

In [ ]:
cic_p = cic_p.astype({'Flow Duration':'float'})
cic_p = cic_p.astype({'Total Fwd Packets':'float'})
cic_p = cic_p.astype({'Total Bwd Packets':'float'})

In [ ]:
# Ton-IoT 데이터셋 크기에 맞게 Sampling

In [ ]:
cic_s = cic_p.sample(n=461043, replace=False, axis=0)

In [ ]:
frames = [cic_s, ton_p]
df = pd.concat(frames)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier()

In [ ]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import time
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# 단일 데이터셋(Ton-IoT)으로 테스트

In [ ]:
X = df[['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Bwd Packets', 'Fwd Packet Length Mean', 'Bwd Packet Length Mean', 'Fwd Packets/s', 'Bwd Packets/s']]
Y = df['Label']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=10, test_size=0.2)

In [ ]:
Z = ton[['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Bwd Packets', 'Fwd Packet Length Mean', 'Bwd Packet Length Mean', 'Fwd Packets/s', 'Bwd Packets/s']]
W = ton['Label']
Z_train, Z_test, W_train, W_test = train_test_split(Z, W, random_state=10, test_size=0.4)

In [ ]:
# 테스트 데이터셋을 Ton-IoT 데이터셋으로 교체

In [ ]:
X_test = Z_test
Y_test = W_test

In [ ]:
start = time.time()
print('program start...')
print()

RF.fit(X_train, Y_train)  # fit() : 훈련 수행
print()

print('prediction:')
y_pred = RF.predict(X_test)  # predict() : 모델 훈련 후 예측
print(y_pred)
print()

print('Score:')
score = RF.score(X_test,Y_test)  # score() : 평가 점수 계산
print(score)

end = time.time()  # 현재 시각 구하기
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')

print("Classifiction Report :")
print(classification_report(Y_test, y_pred))

In [ ]:
# 이종의 데이터셋으로 실험

In [ ]:
X = df[['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Bwd Packets', 'Fwd Packet Length Mean', 'Bwd Packet Length Mean', 'Fwd Packets/s', 'Bwd Packets/s']]
Y = df['Label']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=10, test_size=0.2)

In [ ]:
start = time.time()
print('program start...')
print()

RF.fit(X_train, Y_train)  # fit() : 훈련 수행
print()

print('prediction:')
y_pred = RF.predict(X_test)  # predict() : 모델 훈련 후 예측
print(y_pred)
print()

print('Score:')
score = RF.score(X_test,Y_test)  # score() : 평가 점수 계산
print(score)

end = time.time()  # 현재 시각 구하기
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')

print("Classifiction Report :")
print(classification_report(Y_test, y_pred))